<img src="images/dask_horizontal.svg" align="right" width="30%">

# Table of Contents
* [Bag: Parallel Lists for semi-structured data](#Bag:-Parallel-Lists-for-semi-structured-data)
	* [Creation](#Creation)
	* [Manipulation](#Manipulation)
		* [Exercise: Accounts JSON data](#Exercise:-Accounts-JSON-data)
		* [Basic Queries](#Basic-Queries)
		* [Use `flatten` to de-nest](#Use-flatten-to-de-nest)
		* [Groupby and Foldby](#Groupby-and-Foldby)
		* [`groupby`](#groupby)
		* [`foldby`](#foldby)
		* [Example with account data](#Example-with-account-data)
		* [Exercise: compute total amount per name](#Exercise:-compute-total-amount-per-name)
	* [DataFrames](#DataFrames)
		* [Denormalization](#Denormalization)
	* [Limitations](#Limitations)


# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

*  [Bag Documenation](http://dask.pydata.org/en/latest/bag.html)
*  [Bag API](http://dask.pydata.org/en/latest/bag-api.html)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc..

In [1]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [2]:
# each element is a text file of JSON lines
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))

In [3]:
# Requires `s3fs` library
# each element is a remote CSV text file
b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv')

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

As with `Array` and `DataFrame` objects, operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution.  

Dask.bag uses `dask.multiprocessing.get` by default.  For examples using dask.bag with a distributed cluster see 

*  [Dask.distributed docs](http://distributed.readthedocs.io/en/latest/)
*  [Blogpost analyzing github data on EC2](https://www.continuum.io/content/dask-distributed-and-anaconda-cluster)

In [4]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<map-lam..., npartitions=10>

In [5]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Exercise: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [6]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "George", "transactions": [{"transaction-id": 965, "amount": 680}, {"transaction-id": 1417, "amount": 470}, {"transaction-id": 2967, "amount": 459}, {"transaction-id": 3093, "amount": 134}, {"transaction-id": 3788, "amount": 441}, {"transaction-id": 4178, "amount": 328}, {"transaction-id": 5014, "amount": 436}, {"transaction-id": 5527, "amount": 519}, {"transaction-id": 6855, "amount": 376}, {"transaction-id": 7988, "amount": 216}, {"transaction-id": 8088, "amount": 202}, {"transaction-id": 8558, "amount": 316}, {"transaction-id": 9231, "amount": 458}, {"transaction-id": 9508, "amount": 164}, {"transaction-id": 10106, "amount": 903}, {"transaction-id": 13755, "amount": 299}, {"transaction-id": 17094, "amount": 567}, {"transaction-id": 17425, "amount": 309}, {"transaction-id": 18104, "amount": -103}, {"transaction-id": 20156, "amount": 246}, {"transaction-id": 21535, "amount": 26}, {"transaction-id": 22504, "amount": -60}, {"transaction-id": 24402, "amount": 46}, {"t

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [7]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'George',
  'transactions': [{'amount': 680, 'transaction-id': 965},
   {'amount': 470, 'transaction-id': 1417},
   {'amount': 459, 'transaction-id': 2967},
   {'amount': 134, 'transaction-id': 3093},
   {'amount': 441, 'transaction-id': 3788},
   {'amount': 328, 'transaction-id': 4178},
   {'amount': 436, 'transaction-id': 5014},
   {'amount': 519, 'transaction-id': 5527},
   {'amount': 376, 'transaction-id': 6855},
   {'amount': 216, 'transaction-id': 7988},
   {'amount': 202, 'transaction-id': 8088},
   {'amount': 316, 'transaction-id': 8558},
   {'amount': 458, 'transaction-id': 9231},
   {'amount': 164, 'transaction-id': 9508},
   {'amount': 903, 'transaction-id': 10106},
   {'amount': 299, 'transaction-id': 13755},
   {'amount': 567, 'transaction-id': 17094},
   {'amount': 309, 'transaction-id': 17425},
   {'amount': -103, 'transaction-id': 18104},
   {'amount': 246, 'transaction-id': 20156},
   {'amount': 26, 'transaction-id': 21535},
   {'amount': -60, 'tra

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [8]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 13,
  'name': 'Alice',
  'transactions': [{'amount': 1290, 'transaction-id': 1389},
   {'amount': 1209, 'transaction-id': 1390},
   {'amount': 1248, 'transaction-id': 1469},
   {'amount': 1147, 'transaction-id': 1504},
   {'amount': 1343, 'transaction-id': 2008},
   {'amount': 1096, 'transaction-id': 3507},
   {'amount': 1085, 'transaction-id': 4203},
   {'amount': 1199, 'transaction-id': 4928},
   {'amount': 1173, 'transaction-id': 11859},
   {'amount': 1139, 'transaction-id': 12089},
   {'amount': 1240, 'transaction-id': 15296},
   {'amount': 1202, 'transaction-id': 15560},
   {'amount': 1257, 'transaction-id': 16393},
   {'amount': 1318, 'transaction-id': 17707},
   {'amount': 1205, 'transaction-id': 17890},
   {'amount': 1149, 'transaction-id': 19798},
   {'amount': 1269, 'transaction-id': 20583},
   {'amount': 1235, 'transaction-id': 22742},
   {'amount': 1320, 'transaction-id': 22873},
   {'amount': 1161, 'transaction-id': 22874},
   {'amount': 1272, 'transaction-id': 254

In [9]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'count': 76, 'name': 'Alice'},
 {'count': 477, 'name': 'Alice'},
 {'count': 150, 'name': 'Alice'},
 {'count': 50, 'name': 'Alice'},
 {'count': 120, 'name': 'Alice'})

In [10]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(76, 477, 150, 50, 120)

In [11]:
# Average number of transactions for all of the Alice's
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

169.19454545454545

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [12]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'amount': 1290, 'transaction-id': 1389},
  {'amount': 1209, 'transaction-id': 1390},
  {'amount': 1248, 'transaction-id': 1469},
  {'amount': 1147, 'transaction-id': 1504},
  {'amount': 1343, 'transaction-id': 2008},
  {'amount': 1096, 'transaction-id': 3507},
  {'amount': 1085, 'transaction-id': 4203},
  {'amount': 1199, 'transaction-id': 4928},
  {'amount': 1173, 'transaction-id': 11859},
  {'amount': 1139, 'transaction-id': 12089},
  {'amount': 1240, 'transaction-id': 15296},
  {'amount': 1202, 'transaction-id': 15560},
  {'amount': 1257, 'transaction-id': 16393},
  {'amount': 1318, 'transaction-id': 17707},
  {'amount': 1205, 'transaction-id': 17890},
  {'amount': 1149, 'transaction-id': 19798},
  {'amount': 1269, 'transaction-id': 20583},
  {'amount': 1235, 'transaction-id': 22742},
  {'amount': 1320, 'transaction-id': 22873},
  {'amount': 1161, 'transaction-id': 22874},
  {'amount': 1272, 'transaction-id': 25489},
  {'amount': 1251, 'transaction-id': 26542},
  {'amount': 1158,

In [13]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'amount': 1290, 'transaction-id': 1389},
 {'amount': 1209, 'transaction-id': 1390},
 {'amount': 1248, 'transaction-id': 1469})

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(1290, 1209, 1248)

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

1362.9846223282505

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [17]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Dan', 'Bob']), (5, ['Edith', 'Alice', 'Frank'])]

In [18]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 7, 5, 9])]

In [20]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [27]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [46]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 1100), ('Bob', 750), ('Charlie', 1100), ('Dan', 850), ('Edith', 1300), ('Frank', 848), ('George', 1198), ('Hannah', 700), ('Ingrid', 1000), ('Jerry', 1100), ('Kevin', 1050), ('Laura', 1258), ('Michael', 1100), ('Norbert', 898), ('Oliver', 800), ('Patricia', 1286), ('Quinn', 1000), ('Ray', 900), ('Sarah', 800), ('Tim', 994), ('Ursula', 800), ('Victor', 400), ('Wendy', 990), ('Xavier', 650), ('Yvonne', 1048), ('Zelda', 1000)]
CPU times: user 2min 46s, sys: 3.96 s, total: 2min 50s
Wall time: 3min 22s


In [53]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
incr = lambda tot, _: tot+1
result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 1100), ('Bob', 750), ('Charlie', 1100), ('Dan', 850), ('Edith', 1300), ('Frank', 848), ('George', 1198), ('Hannah', 700), ('Ingrid', 1000), ('Jerry', 1100), ('Kevin', 1050), ('Laura', 1258), ('Michael', 1100), ('Norbert', 898), ('Oliver', 800), ('Patricia', 1286), ('Quinn', 1000), ('Ray', 900), ('Sarah', 800), ('Tim', 994), ('Ursula', 800), ('Victor', 400), ('Wendy', 990), ('Xavier', 650), ('Yvonne', 1048), ('Zelda', 1000)]
CPU times: user 123 ms, sys: 42.8 ms, total: 166 ms
Wall time: 1.51 s


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [47]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [77]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,George,"[{'transaction-id': 965, 'amount': 680}, {'tra..."
1,1,Edith,"[{'transaction-id': 474, 'amount': -240}, {'tr..."
2,2,Norbert,"[{'transaction-id': 1120, 'amount': 211}, {'tr..."
3,3,Dan,"[{'transaction-id': 2, 'amount': 27}, {'transa..."
4,4,Kevin,"[{'transaction-id': 293, 'amount': 550}, {'tra..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [78]:
%time df1.groupby('name')id.count().id.compute().head()

CPU times: user 6.72 s, sys: 699 ms, total: 7.42 s
Wall time: 5.65 s


name
Alice      1100
Bob         750
Charlie    1100
Dan         850
Edith      1300
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [50]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'amount': 680, 'id': 0, 'name': 'George', 'transaction-id': 965},
 {'amount': 470, 'id': 0, 'name': 'George', 'transaction-id': 1417},
 {'amount': 459, 'id': 0, 'name': 'George', 'transaction-id': 2967})

In [72]:
df = transactions.to_dataframe()
df.head()

,amount,id,name,transaction-id
0,680,0,George,965
1,470,0,George,1417
2,459,0,George,2967
3,134,0,George,3093
4,441,0,George,3788


In [74]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 12.5 s, sys: 992 ms, total: 13.5 s
Wall time: 11.5 s


name
Alice       186114
Bob         168882
Charlie     212687
Dan         203297
Edith       261379
Frank       171152
George      211467
Hannah      114087
Ingrid      223403
Jerry       333539
Kevin       216705
Laura       240445
Michael     229052
Norbert     135601
Oliver       99881
Patricia    334445
Quinn       259642
Ray         149855
Sarah       126948
Tim         163784
Ursula      168267
Victor      103947
Wendy       155910
Xavier      103721
Yvonne      156898
Zelda       268892
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.